**извлечение признаков из текста на естественном языке**

токенизация и очистка текста


Евгений Борисов borisov.e@solarl.ru

## библиотеки

In [1]:
import numpy as np
np.set_printoptions(precision=2) # вывод на печать до 2 знака
import pandas as pd
import re
import gzip

In [2]:
pd.options.display.max_colwidth = 200  

## загружаем тексты

In [3]:
# загружаем тексты
df = pd.read_pickle('../data/text/news.pkl.gz')
print('записей:',len(df))

записей: 3196


In [4]:
df.sample(5)

,text,tag
2259,"В Южной Корее на военной базе вблизи города Ульсан прогремел взрыв. Об этом сообщает Yonhap News\n\nПо данным агентства, в результате пострадали 20 человек. Большинство из них получили ожоги разли...",incident
657,"Купите квартиру по цене без отделки, а ремонт получите в подарок\n\n3 декабря 2016 в 9:00\n\n*На правах рекламы\n\nДекабрь — месяц традиционных подарков и сюрпризов. Застройщик жилого квартала «Но...",reclama
1060,От 5 до 50%. Где минчанам искать скидки в декабре\n\n29 ноября 2016 в 14:39\n\nFINANCE.TUT.BY\n\nСтоличные универмаги и торговые сети в декабре будут часто предоставлять скидки на товары. Об акция...,economics
3159,ИА Dv-News (dv-news.com) Выделены средства на ремонт Магаданского дома\nпрестарелых На ремонт Магаданского областного дома-интерната для престарелых\nи инвалидов ?Ветераны Колымы? выделено более м...,social
2949,"Новое поколение Q3 копания Audi строит, что вполне ожидаемо, на модульной платформе MQB концерна Volkswagen. Как утверждает издание Auto Bild, в Ингольштадте собираются воспользоваться всеми преим...",auto


In [7]:
# df['text'].str.len().describe()
df['tag'].drop_duplicates().count()

13

## токенизация и очистка

In [12]:
# with gzip.open('../data/text/stop-nltk.txt.gz','rt',encoding='utf-8') as f: 
#     stopwords = set([ w.strip() for w in  f.read().split() if w.strip() ] )

# print('количество стоп-слов:',len(stopwords))
# sorted(stopwords)

from Stemmer import Stemmer
# pacman -S python-pystemmer
# pip install pystemmer

In [9]:
# from nltk.stem.snowball import SnowballStemmer
# from nltk.corpus import stopwords as nltk_stopwords

# stopwords = set(nltk_stopwords.words('russian') )

In [13]:
%%time 

df['text_clean'] = df['text'].str.lower() # приведение в lowercase

# замена символов-разделителей (-,_) на пробел
df['text_clean'] = df['text_clean'].apply(lambda s: re.sub( r'\W', ' ', s))
df['text_clean'] = df['text_clean'].apply(lambda s: re.sub( r'_', ' ', s))

# замена цифр
df['text_clean'] = df['text_clean'].apply(lambda s: re.sub( r'\b\d+\b', ' ', s))

# делим строки на слова (токенизация)
df['text_clean'] = df['text_clean'].apply(lambda t: [ w.strip() for w in t.split() if len(w.strip())>2 ] )

# удаление лишних слов 
#df['text_clean'] = df['text_clean'].apply(lambda t:[w for w in t if w not in stopwords])

# стемминг, выделение основы слова
df['text_clean'] = df['text_clean'].apply( lambda t: Stemmer('russian').stemWords(t) )
#df['text_clean'] = df['text_clean'].apply(lambda t:[ SnowballStemmer('russian').stem(w) for w in t if w])


CPU times: user 3.52 s, sys: 65.7 ms, total: 3.59 s
Wall time: 3.61 s


In [14]:
df[['text_clean']].sample(3)

,text_clean
1674,"[экс, президент, ссср, миха, горбач, заяв, что, границ, союз, может, появ, нов, объединен, котор, добровольн, войдут, бывш, советск, республик, отмет, что, качеств, возврат, ссср, так, невозможн, ..."
1475,"[ки, декабр, лидер, радикальн, парт, украин, олег, ляшк, раскритикова, надежд, савченк, встреч, глав, луганск, донецк, народн, республик, игор, плотницк, александр, захарченк, суд, текст, опублико..."
1920,"[президент, сша, барак, обам, арендова, офис, здан, всемирн, фонд, дик, природ, wwf, вашингтон, сообща, газет, washington, post, ссылк, источник, заключен, соглашен, аренд, пок, был, объявл, публи..."
